In [32]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
model_id = "microsoft/Phi-3.5-mini-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [9]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out

In [ ]:
model.config

In [11]:
batch_data = ["The world would be a better place if we all", "I am hungry and I want to eat some good"]

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [13]:
model_inputs = tokenizer(batch_data, return_tensors="pt", padding=True).to(device)
model_inputs

{'input_ids': tensor([[32000,   450,  3186,   723,   367,   263,  2253,  2058,   565,   591,
           599],
        [  306,   626,  9074, 14793,   322,   306,   864,   304, 17545,   777,
          1781]], device='cuda:0'), 'attention_mask': tensor([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [30]:
model.eval()
with torch.no_grad():
  model_outputs = model.generate(**model_inputs, max_new_tokens=30, do_sample=True, num_beams=4, num_return_sequences=2)

In [31]:
generated_text = tokenizer.batch_decode(model_outputs, skip_special_tokens=True)
generated_text

["The world would be a better place if we all had a little more compassion and understanding.\n\nSarah: I couldn't agree more. It's important to remember that",
 'The world would be a better place if we all had a little more compassion and understanding.\n\nTheory of Mind Exercises:\n\n1. What would happen if the',
 'I am hungry and I want to eat some good food.\n\n# Answer\nI am hungry and I want to eat some delicious and nutritious food to satisfy my appetite',
 'I am hungry and I want to eat some good food.\n\n# Answer\nI am hungry and I would like to have a delicious meal to satisfy my appetite.']

In [33]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 50,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 To solve the linear equation 2x + 3 = 7, follow these steps:

Step 1: Isolate the variable term (2x) by subtracting 3 from both sides of the equation.
2x +
